In [ ]:
%load_ext dockermagic

# Hive
![Hive](https://hive.apache.org/images/hive_logo_medium.jpg)

- https://hive.apache.org/

## Setup

- version 3.1.3

In [ ]:
%%dockerexec hadoop

# Download package
mkdir -p /opt/pkgs
cd /opt/pkgs
wget -q -c https://downloads.apache.org/hive/hive-3.1.3/apache-hive-3.1.3-bin.tar.gz

# unpack file and create link
tar -zxf apache-hive-3.1.3-bin.tar.gz -C /opt
ln -s /opt/apache-hive-3.1.3-bin /opt/hive

# update envvars.sh
cat >> /opt/envvars.sh << EOF
# Hive
export HIVE_HOME=/opt/hive
export PATH=\${PATH}:\${HIVE_HOME}/bin

EOF

# Fix slf4j
rm /opt/hive/lib/log4j-slf4j-impl-2.17.1.jar

cat /opt/envvars.sh

## Hadoop configuration (for beeline)

- core-site.xml

```xml
<configuration>
...
<property>
  <name>hadoop.proxyuser.hadoop.groups</name>
  <value>*</value>
</property>
<property>
  <name>hadoop.proxyuser.hadoop.hosts</name>
  <value>*</value>
</property>
</configuration>
```

## Hive Metastore

- using local Derby database

### Create directory in HDFS

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfs -mkdir -p /user/hive/warehouse
hdfs dfs -chmod g+w /user/hive/warehouse

### Initialize database

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

mkdir -p $HIVE_HOME/hiveserver2
cd $HIVE_HOME/hiveserver2
$HIVE_HOME/bin/schematool -dbType derby -initSchema 2> /dev/null

### Start hiveserver2

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/hive/hiveserver2
nohup /opt/hive/bin/hive --service hiveserver2 \
--hiveconf hive.security.authorization.createtable.owner.grants=ALL \
--hiveconf hive.root.logger=INFO,console > hiveserver2.out 2>&1 &
echo $! > hiveserver2.pid

## Example

- SF Bay Area Bike Share (https://www.kaggle.com/benhamner/sf-bay-area-bike-share)
- stations.csv and trips.csv

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

mkdir -p /opt/datasets_hive

In [ ]:
%%bash

# copy datasets used by hive examples to hadoop container
docker cp hivedataset.tgz hadoop:/opt/datasets_hive

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets_hive
tar -zxf hivedataset.tgz
rm hivedataset.tgz
ls

hdfs dfs -mkdir -p bikeshare/stations
hdfs dfs -put stations.csv bikeshare/stations
hdfs dfs -mkdir -p bikeshare/trips
hdfs dfs -put trips.csv bikeshare/trips

## Using beeline

In [ ]:
%%dockerwrite hadoop /opt/script.sql

-- configure jobs executor
SET hive.execution.engine=mr;
SET mapreduce.framework.name=yarn;

-- create bikeshare database
CREATE DATABASE bikeshare;
SHOW DATABASES;

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

beeline -n hadoop -u jdbc:hive2://localhost:10000 --silent=true -f /opt/script.sql

In [ ]:
%%dockerwrite hadoop /opt/script.sql

USE bikeshare;

-- create stations table
CREATE EXTERNAL TABLE stations (
    station_id INT,
    name STRING,
    lat DOUBLE,
    long DOUBLE,
    dockcount INT,
    landmark STRING,
    installation STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs:///user/hadoop/bikeshare/stations';

-- create trips table
CREATE EXTERNAL TABLE trips (
    trip_id INT,
    duration INT,
    start_date STRING,
    start_station STRING,
    start_terminal INT,
    end_date STRING,
    end_station STRING,
    end_terminal INT,
    bike_num INT,
    subscription_type STRING,
    zip_code STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs:///user/hadoop/bikeshare/trips';

-- show tables
SHOW TABLES;

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

beeline -n hadoop -u jdbc:hive2://localhost:10000 --silent=true -f /opt/script.sql

In [ ]:
%%dockerwrite hadoop /opt/script.sql

USE bikeshare;

DESCRIBE stations;
DESCRIBE trips;
DESCRIBE FORMATTED stations;
DESCRIBE FORMATTED trips;

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

beeline -n hadoop -u jdbc:hive2://localhost:10000 --silent=true -f /opt/script.sql

In [ ]:
%%dockerwrite hadoop /opt/script.sql

USE bikeshare;

-- query - number of trips per terminal
SELECT start_terminal, start_station, COUNT(1) AS count
FROM trips
GROUP BY start_terminal, start_station
ORDER BY count
DESC LIMIT 10;

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

beeline -n hadoop -u jdbc:hive2://localhost:10000 --silent=true -f /opt/script.sql

In [ ]:
%%dockerwrite hadoop /opt/script.sql

USE bikeshare;

-- query - join between stations and trips
SELECT t.trip_id, t.duration, t.start_date, s.name, s.lat, s.long, s.landmark
FROM stations s
JOIN trips t ON s.station_id = t.start_terminal
LIMIT 10;

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

beeline -n hadoop -u jdbc:hive2://localhost:10000 --silent=true -f /opt/script.sql

## WordCount using Hive

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

mkdir -p /opt/datasets_hive
cd /opt/datasets_hive

wget -q -c https://tinyurl.com/y68jxy7f -O stop-word-list.csv
hdfs dfs -mkdir -p stopwords
hdfs dfs -put stop-word-list.csv stopwords
hdfs dfs -cat stopwords/stop-word-list.csv

# download book "The Complete Works of William Shakespeare, by William Shakespeare" from Gutenberg Project
wget -q -c http://www.gutenberg.org/files/100/100-0.txt -O shakespeare.txt

# create directory in HDFS and put file
hdfs dfs -mkdir -p shakespeare
hdfs dfs -put shakespeare.txt shakespeare
hdfs dfs -ls -h shakespeare

In [ ]:
%%dockerwrite hadoop /opt/wordcount.sql

CREATE TABLE shakespeare_text (line STRING);
LOAD DATA INPATH '/user/hadoop/shakespeare/shakespeare.txt' INTO TABLE shakespeare_text;

CREATE TABLE stopwords (word STRING);
CREATE TABLE tempwords (line STRING);
LOAD DATA INPATH '/user/hadoop/stopwords/stop-word-list.csv' INTO TABLE tempwords;

-- split comma-separated stopwords to rows
INSERT INTO stopwords
SELECT word
FROM tempwords
LATERAL VIEW explode(split(line, ',')) t AS word;
DROP TABLE tempwords;

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

beeline -n hadoop -u jdbc:hive2://localhost:10000 --silent=true -f /opt/wordcount.sql

In [ ]:
%%dockerwrite hadoop /opt/wordcount.sql

SELECT w.word, count(1) AS count
FROM (
    SELECT explode(split(regexp_replace(lower(line), '[^a-z\\s]', ''), '\\s+')) AS word
    FROM shakespeare_text
) w
LEFT OUTER JOIN (
    SELECT lower(trim(word)) AS word
    FROM stopwords
) s ON w.word = s.word
WHERE s.word IS NULL AND w.word != ''
GROUP BY w.word
ORDER BY count DESC
LIMIT 30;

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

beeline -n hadoop -u jdbc:hive2://localhost:10000 --silent=true -f /opt/wordcount.sql

In [ ]:
%%dockerexec hadoop

cd /opt/hive/hiveserver2

# kill hiveserver2
kill $(cat hiveserver2.pid)
rm hiveserver2.pid